In [39]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
import utils
import torchvision
from glob import glob
from torchvision import datasets, models, transforms
from tqdm import tqdm_notebook
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd
import apex.amp as amp

import io
from PIL import Image
import tensorflow as tf


In [10]:
checkpoint = torch.load("checkpoints_299_50/checkpoints_16")

In [12]:
model_conv = torchvision.models.resnet50(pretrained="imagenet").cuda()
data_dir = "data_299/"

for i, param in model_conv.named_parameters():
    param.requires_grad = False

n_class = len(glob(data_dir + "train/*"))

# Since imagenet as 1000 classes , We need to change our last layer according to the number of classes we have,
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, n_class).cuda()

In [42]:
model_conv.load_state_dict(checkpoint["model_state_dict"])
model_conv.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [17]:
train_csv = pd.read_csv("/hdd/kaggle/landmarks/csv/train.csv")

In [18]:
train_csv.head()

,id,url,landmark_id
0,6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820
1,202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169
2,3ad87684c99c06e1,http://upload.wikimedia.org/wikipedia/commons/...,37914
3,e7f70e9c61e66af3,https://upload.wikimedia.org/wikipedia/commons...,102140
4,4072182eddd0100e,https://upload.wikimedia.org/wikipedia/commons...,2474


In [36]:
val_image_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transforms.Compose([transforms.Resize(299), transforms.ToTensor()]))

In [37]:
val_dataloader = torch.utils.data.DataLoader(val_image_dataset, batch_size=128,
                                         shuffle=False, num_workers=4)

In [41]:
# for data in tqdm_notebook(val_dataloader):
for data in val_dataloader:

    inputs, labels = data
    inputs = Variable(inputs.cuda())
    labels = Variable(labels.cuda())
    print("Running inference")
    outputs = model_conv(inputs)
    if type(outputs) == tuple:
        outputs, _ = outputs
    confs, preds = torch.max(outputs.data, 1)
    print(len(confs), len(preds), len(labels))
    print(confs[0], preds[0], labels[0])
    print(confs, preds, labels)
    stop

Running inference
128 128 128
tensor(5.5183, device='cuda:0') tensor(5736, device='cuda:0') tensor(0, device='cuda:0')
tensor([5.5183, 5.6423, 4.9731, 6.0831, 4.2361, 4.6315, 4.8939, 3.8924, 5.2402,
        4.3553, 6.2674, 5.0372, 4.3764, 4.9118, 5.0620, 5.6638, 4.3819, 5.2741,
        7.4727, 5.9367, 5.6695, 5.4090, 4.6617, 6.1171, 4.0485, 4.6464, 4.7053,
        4.2781, 4.7491, 6.4827, 5.5402, 5.1822, 3.8589, 5.4155, 3.8006, 5.1791,
        4.6303, 3.6105, 5.4928, 4.3054, 5.3227, 5.6901, 6.8944, 4.0141, 5.3867,
        3.9643, 4.7282, 5.2347, 4.1324, 6.3667, 4.6912, 4.6557, 7.6732, 4.6460,
        4.9676, 5.7565, 4.6949, 6.0523, 6.5104, 4.3338, 7.8756, 3.9409, 3.5031,
        7.4854, 3.7020, 7.2505, 8.1767, 8.2604, 6.3033, 6.3157, 6.3233, 5.3809,
        6.1483, 4.9785, 4.3489, 3.9248, 5.1976, 5.9655, 7.4782, 5.1292, 5.5689,
        6.0442, 4.5449, 5.2435, 4.7517, 4.3320, 5.1227, 3.5712, 4.4213, 5.0799,
        5.5587, 4.2417, 3.4636, 4.0947, 3.3580, 3.8580, 3.7319, 5.9753, 3.4680,
 

NameError: name 'stop' is not defined